In [ ]:
# Necessary Imports

import requests 
from bs4 import BeautifulSoup
import pandas as pd 
import os 
import nltk 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# Reading the Data 

data = pd.read_excel(r"C:\Users\Admin\OneDrive\Desktop\DATASETS\Blackcoffer\Input.xlsx")
data.head()

,URL_ID,URL
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...
4,Netclan20241021,https://insights.blackcoffer.com/development-o...


In [ ]:
# Data Extraction

for index, row in data.iterrows():
    url = row['URL']
    url_id = row['URL_ID']
    
    header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
    try: 
        response = requests.get(url, headers=header)
    except: 
        print("can't get response of {}".format(url_id))
    try:
        soup = BeautifulSoup(response.content, 'html.parser')
    except:
        print("can't get page of {}".format(url_id))
    try:
        title= soup.find('h1').get_text()
    except:
        print("can't get title of {}".format(url_id))
        continue 

    article = ""
    try: 
        for p in soup.find_all('p'):
            article += p.get_text()
    except:
        print("can't get text of {}". format(url_id))

    file_name = r"C:\Users\Admin\OneDrive\Desktop\DATASETS\Blackcoffer\titletext"  + str(url_id) + '.txt'
    with open(file_name, 'w', encoding="utf-8") as file : 
        file.write(title + '\n' + article)


In [ ]:
# Extracted Data and Other necessary Directories

text_dir  = r"C:\Users\Admin\OneDrive\Desktop\DATASETS\Blackcoffer\TitleText"
stopwords_dir = r"C:\Users\Admin\OneDrive\Desktop\DATASETS\Blackcoffer\StopWords"
sentment_dir = r"C:\Users\Admin\OneDrive\Desktop\DATASETS\Blackcoffer\MasterDictionary"

In [ ]:
# Analysis

from nltk import word_tokenize 
nltk.download('wordnet')
nltk.download('punkt')
stop_words = set()
for files in os.listdir(stopwords_dir):
    try : 
        with open(os.path.join(stopwords_dir,files), 'r', encoding=' ISO-8859-1', errors='ignore') as f:
            stop_words.update(set(f.read().splitlines()))
    except FileNotFoundError : 
         print("Warning:", files, "not found in", stopwords_dir)
          
docs = []
for text_file in os.listdir(text_dir):
    with open(os.path.join(text_dir,text_file), 'r',encoding='ISO-8859-1',errors='ignore') as f :
        text = f.read()
        words = word_tokenize(f.read())
        filtered_text = [word for word in words if word.lower() not in stop_words]
        docs.append(filtered_text)

pos=set()
neg=set()

for files in os.listdir(sentment_dir):
    if files == 'positive.words.txt':
        with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1', errors='ignore') as f :
            pos.update(f.read().splitlines())
    else: 
        with open(os.path.join(sentment_dir,files),'r', encoding = 'ISO-8859-1', errors='ignore') as f :
            neg.update(f.read().splitlines())

positive_words = []
Negative_words = []
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []


for i in range(len(docs)):
    positive_words.append([word for word in docs[i] if word.lower() in pos ])
    Negative_words.append([word for word in docs[i] if word.lower()in neg ])
    positive_score.append(len(positive_words[i]))
    negative_score.append(len(Negative_words[i]))
    polarity_score.append((positive_score[i]- negative_score[i])/ ((len(docs[i]))+ 0.000001))
    subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
avg_sentence_length = []
Percentage_of_Complex_words = []
Fog_Index = []
complex_word_count = []
avg_syllable_word_count = []
stopwords = set(stopwords.words('english'))
def measure(file):
    with open(os.path.join(text_dir, file),'r', encoding='ISO-8859-1', errors='ignore')as f:
        text = f.read()
        text = re.sub(r'[^\w\s.]','',text)
        sentences = text.split('.')
        num_sentences = len(sentences)
        words = [word for word in text.split() if word.lower() not in stopwords]
        num_words = len(words)
        complex_words = []
        for word in words : 
            vowels = 'aeiou'
            syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
            if syllable_count_word > 2 :
                complex_words.append(word)
        syllable_count = 0 
        syllable_words = []
        for word in words :
            if word.endswith('es'):
                word = word[:-2]
            elif word.endswith('ed'):
                word = words[:-2] 
            vowels = 'aeiou'
            syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
            if syllable_count_word >= 1 :
                syllable_words.append(word)
                syllable_count += syllable_count_word

        avg_sentence_len = num_words / num_sentences
        avg_syllable_word_count = syllable_count / len(syllable_words)
        Percent_Complex_words = len(complex_words) / num_words
        Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

        return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words), avg_syllable_word_count
        

for file in os.listdir(text_dir):
    x,y,z,a,b = measure(file)
    avg_sentence_length.append(x)
    Percentage_of_Complex_words.append(y)
    Fog_Index.append(z)
    complex_word_count.append(a)
    avg_syllable_word_count.append(b)

In [6]:
def cleaned_words(file):
    with open(os.path.join(text_dir, file), 'r',encoding='ISO-8859-1',errors='ignore') as f:
        text = f.read()
        text= re.sub(r'[^\w\s]','',text)
        words = [word for word in text.split() if word.lower() not in stopwords]
        length = sum(len(word) for word in words)
        average_word_length = length / len(words)
    return len(words), average_word_length

word_count = []
average_word_length = []
for file in os.listdir(text_dir):
    x,y = cleaned_words(file)
    word_count.append(x)
    average_word_length.append(y)

def count_personal_pronouns(file):
    with open(os.path.join(text_dir, file), 'r',errors='ignore',encoding='ISO-8859-1') as f: 
        text = f.read()
        personal_pronouns = ['I','we', 'my', 'ours','us']
        count = 0 
        for pronoun in personal_pronouns:
            count += len(re.findall(r"\b" + pronoun + r"\b", text))
    return count

pp_count = []
for file in os.listdir(text_dir):
    x = count_personal_pronouns(file)
    pp_count.append(x)
                

In [7]:
output_df = pd.read_excel(r"C:\Users\Admin\OneDrive\Desktop\DATASETS\Blackcoffer\Output Data Structure.xlsx")
len(output_df.columns)

15

In [ ]:
# Output

for i, value in enumerate(positive_score):
    output_df.iloc[:,2] = value

for i, value in enumerate(negative_score):
    output_df.iloc[:,3]= value

for i, value in enumerate(polarity_score):
    output_df.iloc[:,4] = value

for i, value in enumerate(subjectivity_score):
    output_df.iloc[:,5] = value

for i, value in enumerate(avg_sentence_length):
    output_df.iloc[:,6] = value

for i, value in enumerate(Percentage_of_Complex_words):
    output_df.iloc[:,7] = value

for i, value in enumerate(Fog_Index):
    output_df.iloc[:,8] = value

for i, value in enumerate(avg_sentence_length):
    output_df.iloc[:,9] = value

for i, value in enumerate(complex_word_count):
    output_df.iloc[:,10] = value

for i, value in enumerate(word_count):
    output_df.iloc[:,11] = value
    
for i, value in enumerate(avg_syllable_word_count):
    output_df.iloc[:,12] = value
    
for i, value in enumerate(pp_count):
    output_df.iloc[:,13] = value
    
for i, value in enumerate(average_word_length):
    output_df.iloc[:,14] = value
    
    
output_df.to_csv(r"C:\Users\Admin\OneDrive\Desktop\DATASETS\Blackcoffer\Output.csv")